In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

/Users/txie/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/txie/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 6): Library not loaded: @rpath/libpng16.16.dylib
  Referenced from: /Users/txie/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Reason: Incompatible library version: image.so requires version 56.0.0 or later, but libpng16.16.dylib provides version 54.0.0'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
# Hyperparameters
input_size = 28 * 28
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

### 1. Load and Prepare Dataset

In [4]:
# MNIST dataset
train_data = torchvision.datasets.MNIST(root='../../data',
                                        train=True,
                                        transform=transforms.ToTensor(),
                                        download=True)

test_data = torchvision.datasets.MNIST(root='../../data',
                                       train=False,
                                       transform=transforms.ToTensor())

# Data loarder
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_data, 
                                          batch_size=batch_size, 
                                          shuffle=False)


100%|██████████| 9912422/9912422 [00:00<00:00, 39832406.16it/s]


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 47097859.19it/s]


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 22611223.39it/s]


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6952747.73it/s]


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



### 2. Train Model

In [5]:
# Logistic regression
model = nn.Linear(input_size, num_classes)

In [6]:
# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# nn.CrossEntropyLoss() computes softmax internally

In [7]:
# Train model
total_step = len(train_loader)
for epoch in range (num_epochs):
    for i, (images, labels) in enumerate (train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2163
Epoch [1/5], Step [200/600], Loss: 2.0828
Epoch [1/5], Step [300/600], Loss: 1.9868
Epoch [1/5], Step [400/600], Loss: 1.9434
Epoch [1/5], Step [500/600], Loss: 1.8725
Epoch [1/5], Step [600/600], Loss: 1.8046
Epoch [2/5], Step [100/600], Loss: 1.7399
Epoch [2/5], Step [200/600], Loss: 1.7096
Epoch [2/5], Step [300/600], Loss: 1.6315
Epoch [2/5], Step [400/600], Loss: 1.4745
Epoch [2/5], Step [500/600], Loss: 1.4974
Epoch [2/5], Step [600/600], Loss: 1.5087
Epoch [3/5], Step [100/600], Loss: 1.3699
Epoch [3/5], Step [200/600], Loss: 1.3125
Epoch [3/5], Step [300/600], Loss: 1.3246
Epoch [3/5], Step [400/600], Loss: 1.3275
Epoch [3/5], Step [500/600], Loss: 1.1916
Epoch [3/5], Step [600/600], Loss: 1.2511
Epoch [4/5], Step [100/600], Loss: 1.2291
Epoch [4/5], Step [200/600], Loss: 1.2154
Epoch [4/5], Step [300/600], Loss: 1.2129
Epoch [4/5], Step [400/600], Loss: 1.2049
Epoch [4/5], Step [500/600], Loss: 1.1833
Epoch [4/5], Step [600/600], Loss:

### 3. Test Model
In testing, we don't need to compute gradients (for memory efficiency)

In [8]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 82.6500015258789 %


In [9]:
# Save model checkpoint
torch.save(model.state_dict(), "model.ckpt")